In [112]:
import pandas as pd
from sklearn.tree import DecisionTreeClassifier,DecisionTreeRegressor,export_graphviz
import graphviz
import dtreeviz
import pandas as pd
import numpy as np
import pickle
import os
import json
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import GridSearchCV
from sklearn.ensemble import RandomForestRegressor,RandomForestClassifier
from sklearn.metrics import accuracy_score,confusion_matrix
from sklearn import tree
from pypinyin import pinyin, lazy_pinyin, Style
from autogluon.tabular import TabularDataset, TabularPredictor

In [113]:
# autogluon保存路径
model_path = r"F:\cache_data\zone_ana\gl\modle\autogluon_type"
# 检查路径是否存在，否则便创建
if not os.path.exists(model_path):
    os.makedirs(model_path)
else:
    print("文件夹已存在")

文件夹已存在


In [114]:
dataset = pd.read_csv(r"F:\cache_data\zone_ana\gl\train_data\soil_type_train_point.csv")

In [115]:
# # 筛选满足条件的记录
# pr = dataset[(dataset['MZMC'] == "河流冲积物") & (dataset['NEW_TZ'] == "中层砂质灰潮土")]

# # 如果筛选出的记录少于 5 个，我们将修改所有记录
# num_to_change = min(5, len(pr))

# if num_to_change > 0:
#     # 随机选择 num_to_change 个索引
#     random_indices = np.random.choice(pr.index, num_to_change, replace=False)
#     # 修改选中记录的 NEW_TZ 值
#     dataset.loc[random_indices, 'NEW_TZ'] = "厚层砂质灰潮土"

#     print(f"已随机修改 {num_to_change} 条记录的 NEW_TZ 值为 '厚层砂质灰潮土'")
# else:
#     print("没有找到满足条件的记录")

# # 验证修改结果
# modified_records = dataset[(dataset['MZMC'] == "河流冲积物") & (dataset['NEW_TZ'] == "厚层砂质灰潮土")]
# print(f"修改后，符合新条件的记录数：{len(modified_records)}")

In [116]:
dataset["TZ_label"] = dataset.NEW_TZ.astype("category").cat.codes

In [117]:
dataset['MZMCA'] = dataset['MZMCA'].astype('category')
dataset['DLMCA'] = dataset['DLMCA'].astype('category')
dataset['TZ_label'] = dataset['TZ_label'].astype('category')

In [118]:
result = dataset.groupby('TZ_label', observed=True)["NEW_TZ"].apply(lambda x: list(x.unique())).to_dict()
for one_type in result:
    result[one_type] = result[one_type][0]
print(result)

{0: '中层壤质灰潮土', 1: '中层壤质红色石灰土', 2: '中层壤质黄色石灰土', 3: '中层壤质黑色石灰土', 4: '中层暗泥质黄壤', 5: '中层泥质黄壤', 6: '中层泥质黄红壤', 7: '中层灰泥质黄壤', 8: '中层灰泥质黄棕壤', 9: '中层灰泥质黄红壤', 10: '中层砂泥质红壤', 11: '中层砂泥质黄壤', 12: '中层砂泥质黄棕壤', 13: '中层砂质中性紫色土', 14: '中层砂质酸性紫色土', 15: '中层砾壤质黄色石灰土', 16: '中层砾壤质黑色石灰土', 17: '中层硅质红壤', 18: '中层硅质黄壤', 19: '中层硅质黄红壤', 20: '中层红泥质黄壤', 21: '中层黏质黄色石灰土', 22: '中层黏质黑色石灰土', 23: '厚层壤质灰潮土', 24: '厚层壤质红色石灰土', 25: '厚层壤质黄色石灰土', 26: '厚层壤质黑色石灰土', 27: '厚层泥质黄壤', 28: '厚层灰泥质黄壤', 29: '厚层砂质酸性紫色土', 30: '厚层硅质红壤', 31: '厚层硅质黄壤', 32: '厚层硅质黄红壤', 33: '厚层红泥质黄壤', 34: '厚层黏质黄色石灰土', 35: '浅石灰泥田', 36: '渗石灰泥田', 37: '漂石灰泥田', 38: '潮泥田', 39: '潮泥砂田', 40: '石灰泥田', 41: '紫泥田', 42: '腐中层壤质灰潮土', 43: '腐中层壤质红色石灰土', 44: '腐中层壤质黄色石灰土', 45: '腐中层壤质黑色石灰土', 46: '腐中层暗泥质黄壤', 47: '腐中层泥质黄壤', 48: '腐中层泥质黄棕壤', 49: '腐中层灰泥质石灰性粗骨土', 50: '腐中层灰泥质黄壤', 51: '腐中层灰泥质黄棕壤', 52: '腐中层砂泥质红壤', 53: '腐中层砂泥质黄壤', 54: '腐中层砂质中性紫色土', 55: '腐中层砂质石灰性紫色土', 56: '腐中层砂质酸性紫色土', 57: '腐中层砾壤质黄色石灰土', 58: '腐中层砾壤质黑色石灰土', 59: '腐中层砾砂质中性紫色土', 60: '腐中层硅质红壤', 61: '腐中层硅质黄壤', 62: '腐中层硅质黄壤性土', 63: '腐中层硅质黄

In [119]:
# 保存映射关系
with open(r'D:\worker_code\Terrain_Test\data\soil_dict_20241115_gl.json', 'w') as f:
    json.dump(result, f)

In [120]:
# 查看地类类别
tdlylx_df = pd.DataFrame(dataset['DLLBTL'].value_counts())
tdlylx_df

,count
DLLBTL,
自然土,4186
非自然土,2055
水稻土,583


In [121]:
# 查看地类类别
dl_df = pd.DataFrame(dataset['DLMCA'].value_counts())
dl_df

,count
DLMCA,
林地,3920
旱地,1740
水田,583
种植园用地,313
其他土地,255
草地,11
湿地,2


In [122]:
# 查看母质类别
mz_df = pd.DataFrame(dataset['MZMCA'].value_counts())
mz_df

,count
MZMCA,
白云岩,3197
灰岩,2203
砂岩,742
泥(页)岩,250
河流冲积物,140
紫色砂(页)岩,121
第四纪红色黏土,73
砂页岩,68
峨嵋山玄武岩,30


### 砂岩、变余砂岩土种

In [123]:
# 筛选砂岩类水稻土数据
sy_sdt_data = dataset[(dataset['DLLBTL']=='水稻土')
                      & ((dataset['MZMCA'] == '砂岩'))]
pd.unique(sy_sdt_data['NEW_TZ']),#pd.unique(sy_sdt_data['NEW_TS'])

(array(['黄白粉泥田', '黄渗白粉泥田', '黄浅白粉泥田'], dtype=object),)

In [124]:
# 筛砂岩选非自然土数据
sy_fzrt_data = dataset[(dataset['DLLBTL']=='非自然土') 
                       & ((dataset['MZMCA'] == '砂岩'))]
pd.unique(sy_fzrt_data['NEW_TZ']),#pd.unique(sy_fzrt_data['NEW_TS'])

(array(['中层硅质黄壤', '中层硅质黄红壤', '厚层硅质黄红壤', '薄层硅质黄壤', '薄层硅质红壤', '中层硅质红壤',
        '厚层硅质黄壤', '薄层硅质黄红壤', '厚层硅质红壤'], dtype=object),)

In [125]:
# 筛选自然土数据
sy_zrt_data = dataset[(dataset['DLLBTL']=='自然土')
                   & ((dataset['MZMCA'] == '砂岩'))]
pd.unique(sy_zrt_data['NEW_TZ']),#pd.unique(sy_zrt_data['NEW_TS'])

(array(['腐中层硅质黄壤', '腐厚层硅质黄壤', '腐薄层硅质黄壤', '腐中层硅质黄红壤', '腐中层硅质黄壤性土',
        '腐厚层硅质黄红壤', '腐薄层硅质黄壤性土', '腐厚层硅质红壤', '腐中层硅质红壤'], dtype=object),)

### 碳酸岩土种

In [126]:
# 筛选碳酸岩水稻土数据
tsy_sdt_data = dataset[(dataset['DLLBTL']=='水稻土')
                      & ((dataset['MZMCA'] == '白云岩')|(dataset['MZMCA'] == '灰岩'))]
pd.unique(tsy_sdt_data['NEW_TZ']),#pd.unique(tsy_sdt_data['NEW_TS']),

(array(['石灰泥田', '渗石灰泥田', '黄渗灰泥田', '浅石灰泥田', '漂石灰泥田', '黄石灰泥田'], dtype=object),)

In [127]:
# 筛选碳酸岩选非自然土数据
tsy_fzrt_data = dataset[(dataset['DLLBTL']=='非自然土')
                       & ((dataset['MZMCA'] == '白云岩')|(dataset['MZMCA'] == '灰岩'))]

pd.unique(tsy_fzrt_data['NEW_TZ']),#pd.unique(tsy_fzrt_data['NEW_TS']),

(array(['中层壤质黄色石灰土', '厚层壤质黄色石灰土', '薄层壤质黄色石灰土', '中层灰泥质黄壤', '薄层壤质黑色石灰土',
        '中层壤质黑色石灰土', '薄层砾壤质黄色石灰土', '厚层壤质黑色石灰土', '中层砾壤质黄色石灰土', '厚层壤质红色石灰土',
        '中层黏质黄色石灰土', '厚层黏质黄色石灰土', '中层壤质红色石灰土', '薄层灰泥质黄红壤', '薄层壤质红色石灰土',
        '厚层灰泥质黄壤', '中层灰泥质黄红壤', '薄层黏质黄色石灰土', '薄层砾壤质黑色石灰土', '中层砾壤质黑色石灰土',
        '薄层灰泥质黄壤', '中层灰泥质黄棕壤', '薄层灰泥质黄棕壤', '中层黏质黑色石灰土', '薄层砾壤质红色石灰土'],
       dtype=object),)

In [128]:
# 筛选碳酸岩自然土数据

tsy_zrt_data = dataset[(dataset['DLLBTL']=='自然土')
                       & ((dataset['MZMCA'] == '白云岩')|(dataset['MZMCA'] == '灰岩'))]
pd.unique(tsy_zrt_data['NEW_TZ']),#pd.unique(tsy_zrt_data['NEW_TS']),

(array(['腐中层壤质黑色石灰土', '腐中层壤质黄色石灰土', '腐薄层壤质红色石灰土', '腐薄层壤质黄色石灰土', '腐中层灰泥质黄壤',
        '腐薄层砾壤质黄色石灰土', '腐中层壤质红色石灰土', '腐薄层壤质黑色石灰土', '腐薄层砾壤质黑色石灰土',
        '腐中层砾壤质黑色石灰土', '腐薄层黏质黄色石灰土', '腐中层黏质黄色石灰土', '腐薄层灰泥质黄壤',
        '腐薄层灰泥质石灰性粗骨土', '腐中层砾壤质黄色石灰土', '腐厚层壤质黑色石灰土', '腐中层灰泥质黄棕壤',
        '腐薄层灰泥质黄棕壤', '腐厚层灰泥质黄壤', '腐中层灰泥质石灰性粗骨土', '腐厚层黏质黄色石灰土',
        '腐厚层壤质黄色石灰土', '腐中层黏质黑色石灰土', '腐厚层砾壤质黄色石灰土', '腐薄层灰泥质漂洗黄壤'],
       dtype=object),)

### 第四系红粘土

In [129]:
# 筛选第四系红粘土水稻土数据
hnt_sdt_data = dataset[(dataset['DLLBTL']=='水稻土')
                       & ((dataset['MZMCA'] == '第四纪红色黏土'))]
pd.unique(hnt_sdt_data['NEW_TZ']),#pd.unique(hnt_sdt_data['NEW_TS']),

(array(['黄红泥田', '黄渗红泥田'], dtype=object),)

In [130]:
# 筛选第四系红粘土非自然土数据
hnt_fzrt_data = dataset[(dataset['DLLBTL']=='非自然土')
                       & ((dataset['MZMCA'] == '第四纪红色黏土'))]
pd.unique(hnt_fzrt_data['NEW_TZ']),#pd.unique(hnt_fzrt_data['NEW_TS']),

(array(['厚层红泥质黄壤', '中层红泥质黄壤'], dtype=object),)

In [131]:
# 筛选第四系红粘土自然土数据
hnt_zrt_data = dataset[(dataset['DLLBTL']=='自然土')
                       & ((dataset['MZMCA'] == '第四纪红色黏土'))]
pd.unique(hnt_zrt_data['NEW_TZ']),#pd.unique(hnt_zrt_data['NEW_TS']),

(array(['腐厚层红泥质黄壤', '腐中层红泥质黄壤'], dtype=object),)

### 泥(页)岩

In [132]:
# 筛选泥(页)岩水稻土数据
nyy_sdt_data = dataset[(dataset['DLLBTL']=='水稻土')
                       & ((dataset['MZMCA'] == '泥(页)岩'))]
pd.unique(nyy_sdt_data['NEW_TZ']),#pd.unique(nyy_sdt_data['NEW_TS']),

(array(['黄鳝泥田', '黄渗鳝泥田'], dtype=object),)

In [133]:
# 筛选泥(页)岩非自然土数据
nyy_fzrt_data = dataset[(dataset['DLLBTL']=='非自然土')
                       & ((dataset['MZMCA'] == '泥(页)岩'))]
pd.unique(nyy_fzrt_data['NEW_TZ']),#pd.unique(nyy_fzrt_data['NEW_TS']),

(array(['中层泥质黄壤', '厚层泥质黄壤', '中层泥质黄红壤'], dtype=object),)

In [134]:
# 筛选泥(页)岩自然土数据    
nyy_zrt_data = dataset[(dataset['DLLBTL']=='自然土')
                       & ((dataset['MZMCA'] == '泥(页)岩'))]
pd.unique(nyy_zrt_data['NEW_TZ']),#pd.unique(nyy_zrt_data['NEW_TS']),

(array(['腐中层泥质黄壤', '腐厚层泥质黄壤', '腐薄层泥质黄壤', '腐中层泥质黄棕壤', '腐厚层泥质石灰性粗骨土'],
       dtype=object),)

### 紫红色砂页岩	

In [135]:
# 筛选紫红色砂页岩水稻土数据
zhsyy_sdt_data = dataset[(dataset['DLLBTL']=='水稻土')
                       & ((dataset['MZMCA'] == '紫色砂(页)岩'))]
pd.unique(zhsyy_sdt_data['NEW_TZ']),#pd.unique(zhsyy_sdt_data['NEW_TS']),

(array(['酸渗紫泥田', '紫泥田'], dtype=object),)

In [136]:
# 筛选紫红色砂页岩非自然土数据
zhsyy_fzrt_data = dataset[(dataset['DLLBTL']=='非自然土')
                       & ((dataset['MZMCA'] == '紫色砂(页)岩'))]
pd.unique(zhsyy_fzrt_data['NEW_TZ']),#pd.unique(zhsyy_fzrt_data['NEW_TS']),

(array(['中层砂质酸性紫色土', '中层砂质中性紫色土', '厚层砂质酸性紫色土'], dtype=object),)

In [137]:
# 筛选紫红色砂页岩自然土数据
zhsyy_zrt_data = dataset[(dataset['DLLBTL']=='自然土')
                       & ((dataset['MZMCA'] == '紫色砂(页)岩'))]
pd.unique(zhsyy_zrt_data['NEW_TZ']),#pd.unique(zhsyy_zrt_data['NEW_TS']),

(array(['腐中层砂质中性紫色土', '腐中层砾砂质中性紫色土', '腐中层砂质酸性紫色土', '腐中层砂质石灰性紫色土',
        '腐薄层砂质酸性紫色土', '腐薄层砾砂质中性紫色土', '腐厚层砂质酸性紫色土'], dtype=object),)

### 河流冲积物

In [138]:
# 筛选河流冲积物水稻土数据
hlcjw_sdt_data = dataset[(dataset['DLLBTL']=='水稻土')
                       & (dataset['MZMCA'] == '河流冲积物')]
pd.unique(hlcjw_sdt_data['NEW_TZ']),#pd.unique(hlcjw_sdt_data['NEW_TS']),

(array(['潮泥田', '潮泥砂田'], dtype=object),)

In [139]:
# 筛选河流冲积物非自然土数据
hlcjw_fzrt_data = dataset[(dataset['DLLBTL']=='非自然土')
                       & (dataset['MZMCA'] == '河流冲积物')]
pd.unique(hlcjw_fzrt_data['NEW_TZ']),#pd.unique(hlcjw_fzrt_data['NEW_TS']),

(array(['中层壤质灰潮土', '厚层壤质灰潮土'], dtype=object),)

In [140]:
# 筛选河流冲积物自然土数据
hlcjw_zrt_data = dataset[(dataset['DLLBTL']=='自然土')
                       & (dataset['MZMCA'] == '河流冲积物')]
pd.unique(hlcjw_zrt_data['NEW_TZ']),#pd.unique(hlcjw_zrt_data['NEW_TS']),


(array(['腐厚层壤质灰潮土', '腐中层壤质灰潮土'], dtype=object),)

### 砂页岩

In [141]:
# 筛选砂页岩水稻土数据
syy_sdt_data = dataset[(dataset['DLLBTL']=='水稻土')
                       & ((dataset['MZMCA'] == '砂页岩'))]
pd.unique(syy_sdt_data['NEW_TZ']),#pd.unique(syy_sdt_data['NEW_TS']),

(array(['黄渗砂泥田', '黄砂泥田'], dtype=object),)

In [142]:
# 筛选砂页岩非自然土数据
syy_fzrt_data = dataset[(dataset['DLLBTL']=='非自然土')
                       & ((dataset['MZMCA'] == '砂页岩'))]
pd.unique(syy_fzrt_data['NEW_TZ']),#pd.unique(syy_fzrt_data['NEW_TS']),

(array(['中层砂泥质黄壤', '中层砂泥质红壤', '中层砂泥质黄棕壤'], dtype=object),)

In [143]:
# 筛选砂页岩物自然土数据
syy_zrt_data = dataset[(dataset['DLLBTL']=='自然土')
                       & ((dataset['MZMCA'] == '砂页岩'))]
pd.unique(syy_zrt_data['NEW_TZ']),#pd.unique(syy_zrt_data['NEW_TS']),


(array(['腐中层砂泥质黄壤', '腐薄层砂泥质红壤', '腐中层砂泥质红壤', '腐薄层砂泥质黄壤'], dtype=object),)

In [144]:
# 筛选玄武岩自然土数据
xw_zrt_data = dataset[(dataset['DLLBTL']=='自然土')
                       & ((dataset['MZMCA'] == '峨嵋山玄武岩'))]
pd.unique(xw_zrt_data['NEW_TZ']),#pd.unique(xw_zrt_data['NEW_TS']),


(array(['腐薄层暗泥质黄壤', '腐中层暗泥质黄壤'], dtype=object),)

In [145]:
# 筛选峨嵋山玄武岩非自然土数据
xw_fzrt_data = dataset[(dataset['DLLBTL']=='非自然土')
                       & ((dataset['MZMCA'] == '峨嵋山玄武岩'))]
pd.unique(xw_fzrt_data['NEW_TZ']),#pd.unique(xw_fzrt_data['NEW_TS']),



(array(['薄层暗泥质黄壤', '中层暗泥质黄壤'], dtype=object),)

In [99]:
columns_List = [_ for _ in dataset.columns]

In [146]:
features_use = ['analyticalhillshading','aspect','channelnetworkbaselevel','channelnetworkdistance',
 'convergenceindex','dem','dz','dl','etp20221','etp202210','etp202211','etp202212','etp20222','etp20223',
 'etp20224','etp20225','etp20226','etp20227','etp20228','etp20229','etp2022mean','evi','lat','lon','lswi',
 'lsfactor','mndwi','mrrtf','mrvbf','ndmi','ndvi','ndwi','night2022','pca1','pca2','plancurvature',
 'pre20221','pre202210','pre202211','pre202212','pre20222','pre20223','pre20224','pre20225','pre20226',
 'pre20227','pre20228','pre20229','pre2022mean','profilecurvature','relativeslopeposition','savi','slope',
 'gl_slope_101','tmp20221','tmp202210','tmp202211','tmp202212',
 'tmp20222','tmp20223','tmp20224','tmp20225','tmp20226','tmp20227','tmp20228','tmp20229','tmp2022mean',
 'topographicwetnessindex','totalcatchmentarea','valleydepth','vari']

In [147]:

sdt_features = features_use
fzrt_features = features_use
zrt_features = features_use
target = "TZ_label"

### 砂岩数据集

In [148]:
# 获取sdt数据集
sy_sdt_data = sy_sdt_data[sdt_features+[target]]
# 获取非自然土数据集
sy_fzrt_data = sy_fzrt_data[fzrt_features+[target]]
# 获取自然土数据集
sy_zrt_data = sy_zrt_data[zrt_features+[target]]

### 碳酸岩数据集

In [149]:
# 获取sdt数据集
tsy_sdt_data = tsy_sdt_data[sdt_features+[target]]
# 获取非自然土数据集
tsy_fzrt_data = tsy_fzrt_data[fzrt_features+[target]]
# 获取自然土数据集
tsy_zrt_data = tsy_zrt_data[zrt_features+[target]]

### 第四系红粘土数据集

In [150]:
# 获取sdt数据集
hnt_sdt_data = hnt_sdt_data[sdt_features+[target]]
# 获取非自然土数据集
hnt_fzrt_data = hnt_fzrt_data[fzrt_features+[target]]
# 获取自然土数据集
hnt_zrt_data = hnt_zrt_data[zrt_features+[target]]

### 泥(页)岩数据集

In [151]:
# 获取sdt数据集
nyy_sdt_data = nyy_sdt_data[sdt_features+[target]]
# 获取非自然土数据集
nyy_fzrt_data = nyy_fzrt_data[fzrt_features+[target]]
# 获取自然土数据集
nyy_zrt_data = nyy_zrt_data[zrt_features+[target]]

### 紫红色砂页岩数据集

In [152]:
# 获取sdt数据集
zhsyy_sdt_data = zhsyy_sdt_data[sdt_features+[target]]
# 获取非自然土数据集
zhsyy_fzrt_data = zhsyy_fzrt_data[fzrt_features+[target]]
# 获取自然土数据集
zhsyy_zrt_data = zhsyy_zrt_data[zrt_features+[target]]

### 河流冲积物数据集

In [153]:
# 获取sdt数据集
hlcjw_sdt_data = hlcjw_sdt_data[sdt_features+[target]]
# 获取非自然土
hlcjw_fzrt_data = hlcjw_fzrt_data[fzrt_features+[target]]
# 获取自然土
hlcjw_zrt_data = hlcjw_zrt_data[zrt_features+[target]]

### 砂页岩数据集

In [154]:
# 获取sdt数据集
syy_sdt_data = syy_sdt_data[sdt_features+[target]]
# 获取非自然土数据集
syy_fzrt_data = syy_fzrt_data[fzrt_features+[target]]
# 获取自然土数据集
syy_zrt_data = syy_zrt_data[zrt_features+[target]]

In [155]:
# 获取峨嵋山玄武岩数据集
xw_zrt_data = xw_zrt_data[zrt_features+[target]]
# 获取峨嵋山玄武岩非自然土数据集
xw_fzrt_data = xw_fzrt_data[fzrt_features+[target]]

In [156]:
# 查看各个数据集的大小
print('砂岩',sy_sdt_data.shape,sy_fzrt_data.shape,sy_zrt_data.shape)
print('碳酸岩',tsy_sdt_data.shape,tsy_fzrt_data.shape,tsy_zrt_data.shape)
print('第四系红粘土',hnt_sdt_data.shape,hnt_fzrt_data.shape,hnt_zrt_data.shape)
print('泥页岩',nyy_sdt_data.shape,nyy_fzrt_data.shape,nyy_zrt_data.shape)
print('紫红色砂页岩',zhsyy_sdt_data.shape,zhsyy_fzrt_data.shape,zhsyy_zrt_data.shape)
print('河流冲积物',hlcjw_sdt_data.shape,hlcjw_fzrt_data.shape,hlcjw_zrt_data.shape)
print('砂页岩',syy_sdt_data.shape,syy_fzrt_data.shape,syy_zrt_data.shape)
print('峨嵋山玄武岩',xw_zrt_data.shape,xw_fzrt_data.shape)

# 计算总数
total_data = pd.concat([sy_sdt_data,sy_fzrt_data,sy_zrt_data,tsy_sdt_data,tsy_fzrt_data,tsy_zrt_data,hnt_sdt_data,hnt_fzrt_data,hnt_zrt_data,nyy_sdt_data,nyy_fzrt_data,nyy_zrt_data,zhsyy_sdt_data,zhsyy_fzrt_data,zhsyy_zrt_data,hlcjw_sdt_data,hlcjw_fzrt_data,hlcjw_zrt_data,syy_sdt_data,syy_fzrt_data,syy_zrt_data,xw_zrt_data,xw_fzrt_data])
total_data.shape,dataset.shape[0]


砂岩 (64, 72) (331, 72) (347, 72)
碳酸岩 (368, 72) (1489, 72) (3543, 72)
第四系红粘土 (22, 72) (25, 72) (26, 72)
泥页岩 (24, 72) (83, 72) (143, 72)
紫红色砂页岩 (19, 72) (33, 72) (69, 72)
河流冲积物 (75, 72) (53, 72) (12, 72)
砂页岩 (11, 72) (25, 72) (32, 72)
峨嵋山玄武岩 (14, 72) (16, 72)


((6824, 72), 6824)

In [158]:
# 确定标签
label = "TZ_label"
eval_metric = 'f1_weighted'
problem_type = 'multiclass'
# # 10折交叉验证
# cv_num = 10
# 初始化模型和超参数
hyperparameters={
	'NN_TORCH': {},
	'FASTAI': {},
	'RF': [{'criterion': 'gini', 'ag_args': {'name_suffix': 'Gini', 'problem_types': ['binary', 'multiclass']}}, {'criterion': 'entropy', 'ag_args': {'name_suffix': 'Entr', 'problem_types': ['binary', 'multiclass']}}, {'criterion': 'squared_error', 'ag_args': {'name_suffix': 'MSE', 'problem_types': ['regression', 'quantile']}}],
	'XT': [{'criterion': 'gini', 'ag_args': {'name_suffix': 'Gini', 'problem_types': ['binary', 'multiclass']}}, {'criterion': 'entropy', 'ag_args': {'name_suffix': 'Entr', 'problem_types': ['binary', 'multiclass']}}, {'criterion': 'squared_error', 'ag_args': {'name_suffix': 'MSE', 'problem_types': ['regression', 'quantile']}}]
}


### 模型训练

In [159]:
def train_model(train_data,label,eval_metric,hyperparameters,problem_type,model_master,model_branch,model_root):
    # 训练model
    temp_master_path =os.path.join(model_root,model_master)
    temp_branch_path = os.path.join(temp_master_path,model_branch)
    # 检查路径是否存在，否则便创建
    if not os.path.exists(temp_branch_path):
        os.makedirs(temp_branch_path)
    else:
        print("文件夹已存在")
    # 执行训练
    train_predictor = TabularPredictor(label=label,path=temp_branch_path,eval_metric=eval_metric,problem_type=problem_type).fit(train_data,hyperparameters=hyperparameters)
    train_predictor.fit_summary()
    return train_predictor.model_best


### 模型训练

In [160]:
# 砂岩
sy_sdt_predictor = train_model(sy_sdt_data,label=label,eval_metric=eval_metric,hyperparameters=hyperparameters,problem_type=problem_type,model_master='sy',model_branch='sdt',model_root=model_path) 
sy_fzrt_predictor = train_model(sy_fzrt_data,label=label,eval_metric=eval_metric,hyperparameters=hyperparameters,problem_type=problem_type,model_master='sy',model_branch='fzrt',model_root=model_path) 
sy_zrt_predictor = train_model(sy_zrt_data,label=label,eval_metric=eval_metric,hyperparameters=hyperparameters,problem_type=problem_type,model_master='sy',model_branch='zrt',model_root=model_path) 
# 碳酸岩
tsy_sdt_predictor = train_model(tsy_sdt_data,label=label,eval_metric=eval_metric,hyperparameters=hyperparameters,problem_type=problem_type,model_master='tsy',model_branch='sdt',model_root=model_path) 
tsy_fzrt_predictor = train_model(tsy_fzrt_data,label=label,eval_metric=eval_metric,hyperparameters=hyperparameters,problem_type=problem_type,model_master='tsy',model_branch='fzrt',model_root=model_path) 
tsy_zrt_predictor = train_model(tsy_zrt_data,label=label,eval_metric=eval_metric,hyperparameters=hyperparameters,problem_type=problem_type,model_master='tsy',model_branch='zrt',model_root=model_path) 
# 第四系红粘土
hnt_sdt_predictor = train_model(hnt_sdt_data,label=label,eval_metric=eval_metric,hyperparameters=hyperparameters,problem_type=problem_type,model_master='hnt',model_branch='sdt',model_root=model_path) 
hnt_fzrt_predictor = train_model(hnt_fzrt_data,label=label,eval_metric=eval_metric,hyperparameters=hyperparameters,problem_type=problem_type,model_master='hnt',model_branch='fzrt',model_root=model_path) 
hnt_zrt_predictor = train_model(hnt_zrt_data,label=label,eval_metric=eval_metric,hyperparameters=hyperparameters,problem_type=problem_type,model_master='hnt',model_branch='zrt',model_root=model_path) 
# 泥页岩
nyy_sdt_predictor = train_model(nyy_sdt_data,label=label,eval_metric=eval_metric,hyperparameters=hyperparameters,problem_type=problem_type,model_master='nyy',model_branch='sdt',model_root=model_path) 
nyy_fzrt_predictor = train_model(nyy_fzrt_data,label=label,eval_metric=eval_metric,hyperparameters=hyperparameters,problem_type=problem_type,model_master='nyy',model_branch='fzrt',model_root=model_path) 
nyy_zrt_predictor = train_model(nyy_zrt_data,label=label,eval_metric=eval_metric,hyperparameters=hyperparameters,problem_type=problem_type,model_master='nyy',model_branch='zrt',model_root=model_path) 
# 紫红色砂页岩
zhsyy_sdt_predictor = train_model(zhsyy_sdt_data,label=label,eval_metric=eval_metric,hyperparameters=hyperparameters,problem_type=problem_type,model_master='zhsyy',model_branch='sdt',model_root=model_path) 
zhsyy_fzrt_predictor = train_model(zhsyy_fzrt_data,label=label,eval_metric=eval_metric,hyperparameters=hyperparameters,problem_type=problem_type,model_master='zhsyy',model_branch='fzrt',model_root=model_path) 
zhsyy_zrt_predictor = train_model(zhsyy_zrt_data,label=label,eval_metric=eval_metric,hyperparameters=hyperparameters,problem_type=problem_type,model_master='zhsyy',model_branch='zrt',model_root=model_path) 
# 河流冲积物
hlcjw_sdt_predictor = train_model(hlcjw_sdt_data,label=label,eval_metric=eval_metric,hyperparameters=hyperparameters,problem_type=problem_type,model_master='hlcjw',model_branch='sdt',model_root=model_path)
hlcjw_fzrt_predictor = train_model(hlcjw_fzrt_data,label=label,eval_metric=eval_metric,hyperparameters=hyperparameters,problem_type=problem_type,model_master='hlcjw',model_branch='fzrt',model_root=model_path)
hlcjw_zrt_predictor = train_model(hlcjw_zrt_data,label=label,eval_metric=eval_metric,hyperparameters=hyperparameters,problem_type=problem_type,model_master='hlcjw',model_branch='zrt',model_root=model_path)
# 砂页岩
syy_sdt_predictor = train_model(syy_sdt_data,label=label,eval_metric=eval_metric,hyperparameters=hyperparameters,problem_type=problem_type,model_master='syy',model_branch='sdt',model_root=model_path) 
syy_fzrt_predictor = train_model(syy_fzrt_data,label=label,eval_metric=eval_metric,hyperparameters=hyperparameters,problem_type=problem_type,model_master='syy',model_branch='fzrt',model_root=model_path) 
syy_zrt_predictor = train_model(syy_zrt_data,label=label,eval_metric=eval_metric,hyperparameters=hyperparameters,problem_type=problem_type,model_master='syy',model_branch='zrt',model_root=model_path) 
# 峨嵋山玄武岩
xw_zrt_predictor = train_model(xw_zrt_data,label=label,eval_metric=eval_metric,hyperparameters=hyperparameters,problem_type=problem_type,model_master='xw',model_branch='zrt',model_root=model_path) 
xw_fzrt_predictor = train_model(xw_fzrt_data,label=label,eval_metric=eval_metric,hyperparameters=hyperparameters,problem_type=problem_type,model_master='xw',model_branch='fzrt',model_root=model_path) 

No presets specified! To achieve strong results with AutoGluon, it is recommended to use the available presets.
	Recommended Presets (For more details refer to https://auto.gluon.ai/stable/tutorials/tabular/tabular-essentials.html#presets):
	presets='best_quality'   : Maximize accuracy. Default time_limit=3600.
	presets='high_quality'   : Strong accuracy with fast inference speed. Default time_limit=3600.
	presets='good_quality'   : Good accuracy with very fast inference speed. Default time_limit=3600.
	presets='medium_quality' : Fast training time, ideal for initial prototyping.
Beginning AutoGluon training ...
AutoGluon will save models to "F:\cache_data\zone_ana\gl\modle\autogluon_type\sy\sdt"
=================== System Info ===================
AutoGluon Version:  1.1.0
Python Version:     3.10.11
Operating System:   Windows
Platform Machine:   AMD64
Platform Version:   10.0.22631
CPU Count:          20
Memory Avail:       45.25 GB / 63.81 GB (70.9%)
Disk Space Avail:   336.52 GB / 

*** Summary of fit() ***
Estimated performance of each model:
                 model  score_val  eval_metric  pred_time_val  fit_time  pred_time_val_marginal  fit_time_marginal  stack_level  can_infer  fit_order
0  WeightedEnsemble_L2   0.697436  f1_weighted       0.001263  2.619971                0.000000           0.087480            2       True          7
1      NeuralNetFastAI   0.697436  f1_weighted       0.001263  2.532491                0.001263           2.532491            1       True          1
2     RandomForestGini   0.644444  f1_weighted       0.035232  0.364939                0.035232           0.364939            1       True          2
3       ExtraTreesEntr   0.585219  f1_weighted       0.016693  0.341561                0.016693           0.341561            1       True          5
4       ExtraTreesGini   0.585219  f1_weighted       0.034391  0.355829                0.034391           0.355829            1       True          4
5     RandomForestEntr   0.585219  f1_

No presets specified! To achieve strong results with AutoGluon, it is recommended to use the available presets.
	Recommended Presets (For more details refer to https://auto.gluon.ai/stable/tutorials/tabular/tabular-essentials.html#presets):
	presets='best_quality'   : Maximize accuracy. Default time_limit=3600.
	presets='high_quality'   : Strong accuracy with fast inference speed. Default time_limit=3600.
	presets='good_quality'   : Good accuracy with very fast inference speed. Default time_limit=3600.
	presets='medium_quality' : Fast training time, ideal for initial prototyping.
Beginning AutoGluon training ...
AutoGluon will save models to "F:\cache_data\zone_ana\gl\modle\autogluon_type\sy\fzrt"
=================== System Info ===================
AutoGluon Version:  1.1.0
Python Version:     3.10.11
Operating System:   Windows
Platform Machine:   AMD64
Platform Version:   10.0.22631
CPU Count:          20
Memory Avail:       45.03 GB / 63.81 GB (70.6%)
Disk Space Avail:   336.51 GB /

Plot summary of models saved to file: F:\cache_data\zone_ana\gl\modle\autogluon_type\sy\sdtSummaryOfModels.html
*** End of fit() summary ***


	0.6633	 = Validation score   (f1_weighted)
	0.34s	 = Training   runtime
	0.01s	 = Validation runtime
Fitting model: RandomForestGini ...
	0.7604	 = Validation score   (f1_weighted)
	0.47s	 = Training   runtime
	0.03s	 = Validation runtime
Fitting model: RandomForestEntr ...
	0.7058	 = Validation score   (f1_weighted)
	0.51s	 = Training   runtime
	0.03s	 = Validation runtime
Fitting model: ExtraTreesGini ...
	0.7804	 = Validation score   (f1_weighted)
	0.48s	 = Training   runtime
	0.03s	 = Validation runtime
Fitting model: ExtraTreesEntr ...
	0.7735	 = Validation score   (f1_weighted)
	0.48s	 = Training   runtime
	0.03s	 = Validation runtime
Fitting model: NeuralNetTorch ...
	0.6452	 = Validation score   (f1_weighted)
	1.03s	 = Training   runtime
	0.02s	 = Validation runtime
Fitting model: WeightedEnsemble_L2 ...
	Ensemble Weights: {'ExtraTreesGini': 0.667, 'ExtraTreesEntr': 0.333}
	0.7935	 = Validation score   (f1_weighted)
	0.09s	 = Training   runtime
	0.0s	 = Validation runtime
Auto

...

*** Summary of fit() ***
Estimated performance of each model:
                 model  score_val  eval_metric  pred_time_val  fit_time  pred_time_val_marginal  fit_time_marginal  stack_level  can_infer  fit_order
0       ExtraTreesGini   0.847619  f1_weighted       0.030893  0.578680                0.030893           0.578680            1       True          4
1  WeightedEnsemble_L2   0.847619  f1_weighted       0.031895  0.669678                0.001001           0.090997            2       True          7
2       ExtraTreesEntr   0.847619  f1_weighted       0.032691  0.542871                0.032691           0.542871            1       True          5
3     RandomForestGini   0.714286  f1_weighted       0.036530  0.397843                0.036530           0.397843            1       True          2
4     RandomForestEntr   0.714286  f1_weighted       0.048022  0.683750                0.048022           0.683750            1       True          3
5       NeuralNetTorch   0.647619  f1_

	1.0	 = Validation score   (f1_weighted)
	0.56s	 = Training   runtime
	0.05s	 = Validation runtime
Fitting model: RandomForestEntr ...
	1.0	 = Validation score   (f1_weighted)
	0.48s	 = Training   runtime
	0.03s	 = Validation runtime
Fitting model: ExtraTreesGini ...
	1.0	 = Validation score   (f1_weighted)
	0.47s	 = Training   runtime
	0.03s	 = Validation runtime
Fitting model: ExtraTreesEntr ...
	1.0	 = Validation score   (f1_weighted)
	0.46s	 = Training   runtime
	0.03s	 = Validation runtime
Fitting model: NeuralNetFastAI ...
No improvement since epoch 0: early stopping
	0.6667	 = Validation score   (f1_weighted)
	0.11s	 = Training   runtime
	0.01s	 = Validation runtime
Fitting model: NeuralNetTorch ...
		float division by zero
Detailed Traceback:
Traceback (most recent call last):
  File "d:\worker_code\.venvgis\lib\site-packages\autogluon\core\trainer\abstract_trainer.py", line 1904, in _train_and_save
    model = self._train_single(X, y, model, X_val, y_val, total_resources=total

*** Summary of fit() ***
Estimated performance of each model:
                 model  score_val  eval_metric  pred_time_val  fit_time  pred_time_val_marginal  fit_time_marginal  stack_level  can_infer  fit_order
0       ExtraTreesEntr   1.000000  f1_weighted       0.032125  0.463977                0.032125           0.463977            1       True          4
1     RandomForestEntr   1.000000  f1_weighted       0.033416  0.476077                0.033416           0.476077            1       True          2
2       ExtraTreesGini   1.000000  f1_weighted       0.034787  0.474205                0.034787           0.474205            1       True          3
3     RandomForestGini   1.000000  f1_weighted       0.046571  0.557577                0.046571           0.557577            1       True          1
4  WeightedEnsemble_L2   1.000000  f1_weighted       0.048239  0.644671                0.001668           0.087094            2       True          6
5      NeuralNetFastAI   0.666667  f1_

	1.0	 = Validation score   (f1_weighted)
	0.53s	 = Training   runtime
	0.06s	 = Validation runtime
Fitting model: RandomForestEntr ...
	1.0	 = Validation score   (f1_weighted)
	0.51s	 = Training   runtime
	0.03s	 = Validation runtime
Fitting model: ExtraTreesGini ...
	1.0	 = Validation score   (f1_weighted)
	0.49s	 = Training   runtime
	0.05s	 = Validation runtime
Fitting model: ExtraTreesEntr ...
	1.0	 = Validation score   (f1_weighted)
	0.49s	 = Training   runtime
	0.03s	 = Validation runtime
Fitting model: NeuralNetFastAI ...
No improvement since epoch 0: early stopping
	0.1	 = Validation score   (f1_weighted)
	0.13s	 = Training   runtime
	0.0s	 = Validation runtime
Fitting model: NeuralNetTorch ...
		float division by zero
Detailed Traceback:
Traceback (most recent call last):
  File "d:\worker_code\.venvgis\lib\site-packages\autogluon\core\trainer\abstract_trainer.py", line 1904, in _train_and_save
    model = self._train_single(X, y, model, X_val, y_val, total_resources=total_res

*** Summary of fit() ***
Estimated performance of each model:
                 model  score_val  eval_metric  pred_time_val  fit_time  pred_time_val_marginal  fit_time_marginal  stack_level  can_infer  fit_order
0       ExtraTreesEntr        1.0  f1_weighted       0.032511  0.488801                0.032511           0.488801            1       True          4
1     RandomForestEntr        1.0  f1_weighted       0.033461  0.514349                0.033461           0.514349            1       True          2
2       ExtraTreesGini        1.0  f1_weighted       0.049096  0.491439                0.049096           0.491439            1       True          3
3     RandomForestGini        1.0  f1_weighted       0.060914  0.528594                0.060914           0.528594            1       True          1
4  WeightedEnsemble_L2        1.0  f1_weighted       0.061913  0.605695                0.000999           0.077101            2       True          6
5      NeuralNetFastAI        0.1  f1_